In [272]:
%pip install rubik-cube numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [273]:
from rubik.cube import Cube

def colored_cube(cube: Cube):
  cube_str = ""
  for char in cube.__str__():
    match (char):
      case "O":
        cube_str += "\033[38;5;214mO\033[0m"
      case "Y":
        cube_str += "\033[38;5;11mY\033[0m"
      case "W":
        cube_str += "\033[38;5;15mW\033[0m"
      case "G":
        cube_str += "\033[38;5;10mG\033[0m"
      case "B":
        cube_str += "\033[38;5;12mB\033[0m"
      case "R":
        cube_str += "\033[38;5;9mR\033[0m"
      case _:
        cube_str += char
  
  return cube_str

c = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")

print(colored_cube(c))

    OOO
    OOO
    OOO
YYY WWW GGG BBB
YYY WWW GGG BBB
YYY WWW GGG BBB
    RRR
    RRR
    RRR


In [274]:
import random

SOLVED_CUBE_STR = "OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR"
MOVES = ["L", "R", "U", "D", "F", "B", "M", "E", "S"]

def random_cube(num_moves: int = 1):
    """
    :return: A new scrambled Cube
    """
    scramble_moves = " ".join(random.choices(MOVES, k=num_moves))
    c = Cube(SOLVED_CUBE_STR)
    c.sequence(scramble_moves)
    return c

print(colored_cube(random_cube(1)))

    OOO
    OOO
    OOO
YYY WWW GGG BBB
YYY WWW GGG BBB
BBB YYY WWW GGG
    RRR
    RRR
    RRR


In [275]:
c = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")
# print(c._color_list())

def cube_hash(cube: Cube):
  res = 0
  res_list=[]
  for piece in cube._color_list():
    # O: 1, Y: 2, W: 3, G: 4, B: 5, R: 6
    match (piece):
      case "O":
        res_list.append(1)
      case "Y":
        res_list.append(2)
      case "W":
        res_list.append(3)
      case "G":
        res_list.append(4)
      case "B":
        res_list.append(5)
      case "R":
        res_list.append(6)
  
  for i in range(6):
      res *= 9
      res += res_list[i]
  
  return res

c = random_cube(100)
print(colored_cube(c))
print("Cube Hash:", cube_hash(c))



    YGW
    WGG
    WYB
RRG OWG RRY ROB
BOY OWY BRO WBO
ORG RBB GWO WRB
    WGO
    YYG
    YBY
Cube Hash: 146812


In [276]:
import numpy as np

class Agent:
    def __init__(self, alpha=0.9, gamma=0.95, q_init=0.6):
        # alpha: The learning rate needs to be larger than 0 and smaller than 1
        # gamma: The reward discount. Needs to be larger than 0  and should be smaller than 1. Values close to 1 should work best.
        # q_init: The initial q values for each action and state.
        
        self.q = {}  # type: Dict[int, np.ndarray]
        self.action_history = []  # type: List[(int, int)]
        self.learning_rate = alpha
        self.value_discount = gamma
        self.q_init_val = q_init
        super().__init__()

    def get_q(self, hash_value: int) -> np.ndarray:
        # Returns the q values for the state with hash value of cube
        # hash_value: The hash value of the board state for which the q values should be returned
        # return: List of q values for the input state hash.

        #
        # Build Q table in a lazy manner, only adding a state when it is actually used for the first time
        #
        if hash_value in self.q:
            qvals = self.q[hash_value]
        else:
            qvals = np.full(9, self.q_init_val)
            self.q[hash_value] = qvals

        return qvals

    def get_action(self, cube: Cube) -> int:
        # Return the next action given the cube `board` cube on the current Q values
        # cube: The current cube state
        # return: The next action based on the current Q values for the input state
        
        hash_value = cube_hash(cube)  # type: int
        qvals = self.get_q(hash_value)  # type: np.ndarray
        m = np.argmax(qvals)  # type: int
        return m

    def action(self, cube: Cube) -> bool:
        # Makes a action to solve the cube
        # cube: The cube to make a action on
        # return: Bool whether the cube is solved

        m = self.get_action(cube)
        self.action_history.append((cube_hash(cube), m))
        cube.sequence(MOVES[m])
        solved = cube.is_solved()
        return solved

    def update_q(self, moves: int = 0, result: float = None):
        # Gets called after the game has finished. Will update the current Q function based on the actions.

        if(result == None):
            result =  len(self.action_history) / moves

        self.action_history.reverse()
        next_max = -1.0  # type: float

        for a in self.action_history:
            qvals = self.get_q(a[0])
            if next_max < 0:  # First time through the loop
                qvals[a[1]] = result
            else:
                qvals[a[1]] = qvals[a[1]] * (
                            1.0 - self.learning_rate) + self.learning_rate * self.value_discount * next_max

            next_max = max(qvals)

    def new_agent(self):
        self.action_history = []

In [277]:
def try_to_solve(cube: Cube, agent: Agent, num_moves: int, cancle_after: int) -> bool:
    agent.new_agent()
    
    for _ in range(cancle_after):
      solved = agent.action(cube)
      if solved:
        agent.update_q(num_moves)
        return True

    agent.update_q(num_moves, -1.0)
    return False

In [286]:
def train(agent: Agent, num_moves: int = 1, cancle_after: int = 1, num_games: int = 10, silent: bool = False):
  solved_count=0
  for _ in range(num_games):
    cube = random_cube(num_moves)
    solved = try_to_solve(cube, agent, num_moves, cancle_after)
    if(solved):
      solved_count+=1
    # print(solved)

  print(solved_count)
agent = Agent()
train(agent, 1, 3, 1000)

106
